In [516]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import requests


print('Libraries imported.')

Libraries imported.


In [517]:
import numpy as np 
import pandas as pd
from scipy import ndimage 
from scipy.cluster import hierarchy 
from scipy.spatial import distance_matrix 
from matplotlib import pyplot as plt 
from sklearn import manifold, datasets 
from sklearn.cluster import AgglomerativeClustering 
from sklearn.datasets.samples_generator import make_blobs 
%matplotlib inline

In [518]:
csv_path='https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df = pd.read_csv(csv_path,encoding='latin1')
print('Data loaded')

Data loaded


In [519]:
df.head()

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No Designat

In [520]:
Neighborhood = list(df.columns.values)
Neighborhood = Neighborhood[5:]
print(Neighborhood)

['Agincourt North', 'Agincourt South-Malvern West', 'Alderwood', 'Annex', 'Banbury-Don Mills', 'Bathurst Manor', 'Bay Street Corridor', 'Bayview Village', 'Bayview Woods-Steeles', 'Bedford Park-Nortown', 'Beechborough-Greenbrook', 'Bendale', 'Birchcliffe-Cliffside', 'Black Creek', 'Blake-Jones', 'Briar Hill-Belgravia', 'Bridle Path-Sunnybrook-York Mills', 'Broadview North', 'Brookhaven-Amesbury', 'Cabbagetown-South St. James Town', 'Caledonia-Fairbank', 'Casa Loma', 'Centennial Scarborough', 'Church-Yonge Corridor', 'Clairlea-Birchmount', 'Clanton Park', 'Cliffcrest', 'Corso Italia-Davenport', 'Danforth', 'Danforth East York', 'Don Valley Village', 'Dorset Park', 'Dovercourt-Wallace Emerson-Junction', 'Downsview-Roding-CFB', 'Dufferin Grove', 'East End-Danforth', 'Edenbridge-Humber Valley', 'Eglinton East', 'Elms-Old Rexdale', 'Englemount-Lawrence', 'Eringate-Centennial-West Deane', 'Etobicoke West Mall', 'Flemingdon Park', 'Forest Hill North', 'Forest Hill South', 'Glenfield-Jane Heig

In [521]:
dfToronto = pd.DataFrame(index=Neighborhood, columns=["Population_2016","Income_2016"])
dfToronto.head()

,Population_2016,Income_2016
Agincourt North,NaN,NaN
Agincourt South-Malvern West,NaN,NaN
Alderwood,NaN,NaN
Annex,NaN,NaN
Banbury-Don Mills,NaN,NaN


In [522]:
# Population_2016 = Population, 2016
# Income_2016 = Total income: Average amount ($)

for index, row in dfToronto.iterrows():
    dfToronto.at[index, 'Population_2016'] = df[index][2].replace(',', '.')
    dfToronto.at[index, 'Income_2016'] = df[index][2264].replace(',', '.')
    

dfToronto.sort_values('Income_2016')

,Population_2016,Income_2016
St.Andrew-Windfields,17.812,100.516
Edenbridge-Humber Valley,15.535,101.551
Lawrence Park North,14.607,111.730
Annex,30.526,112.766
Yonge-St.Clair,12.528,114.174
Bedford Park-Nortown,23.236,123.077
Leaside-Bennington,16.828,125.564
Kingsway South,9.271,144.642
Casa Loma,10.968,165.047
Lawrence Park South,15.179,169.203


In [523]:
dfToronto.head()

,Population_2016,Income_2016
Agincourt North,29.113,30.414
Agincourt South-Malvern West,23.757,31.825
Alderwood,12.054,47.709
Annex,30.526,112.766
Banbury-Don Mills,27.695,67.757


In [524]:
dfToronto.columns

Index(['Population_2016', 'Income_2016'], dtype='object')

In [525]:
modified = dfToronto.reset_index()
print("Modified Dataframe : ")
print(modified)

Modified Dataframe : 
                                   index Population_2016 Income_2016
0                        Agincourt North          29.113      30.414
1           Agincourt South-Malvern West          23.757      31.825
2                              Alderwood          12.054      47.709
3                                  Annex          30.526     112.766
4                      Banbury-Don Mills          27.695      67.757
5                         Bathurst Manor          15.873      45.936
6                    Bay Street Corridor          25.797      56.526
7                        Bayview Village          21.396      52.035
8                  Bayview Woods-Steeles          13.154      47.246
9                   Bedford Park-Nortown          23.236     123.077
10               Beechborough-Greenbrook           6.577      33.829
11                               Bendale          29.960      33.256
12                 Birchcliffe-Cliffside          22.291      54.360
13          

In [526]:
dfToronto.reset_index(inplace=True)

In [527]:
dfToronto.count()

index              140
Population_2016    140
Income_2016        140
dtype: int64

In [528]:
dfToronto.head()

,index,Population_2016,Income_2016
0,Agincourt North,29.113,30.414
1,Agincourt South-Malvern West,23.757,31.825
2,Alderwood,12.054,47.709
3,Annex,30.526,112.766
4,Banbury-Don Mills,27.695,67.757


In [529]:
dfToronto.columns

Index(['index', 'Population_2016', 'Income_2016'], dtype='object')

In [530]:
dfToronto.rename(columns={'index':'Neighborhood',
                          'Population_2016':'Population_2016',
                          'Income_2016':'Income_2016'}, 
                 inplace=True)

In [531]:
dfToronto.columns

Index(['Neighborhood', 'Population_2016', 'Income_2016'], dtype='object')

In [532]:
dfToronto.head()

,Neighborhood,Population_2016,Income_2016
0,Agincourt North,29.113,30.414
1,Agincourt South-Malvern West,23.757,31.825
2,Alderwood,12.054,47.709
3,Annex,30.526,112.766
4,Banbury-Don Mills,27.695,67.757


In [533]:
List_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(List_url).text

In [534]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [535]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [536]:
df.count()

Postalcode      180
Borough         180
Neighborhood    180
dtype: int64

In [537]:
df=df[df['Borough']!='Not assigned']
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [538]:
df_merge = pd.merge(df, temp_df, on='Postalcode')
df_merge.drop(['Neighborhood'],axis=1,inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [539]:
df_merge.count()

Postalcode      103
Borough         103
Neighborhood    103
dtype: int64

In [540]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [541]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

geo_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [542]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [543]:
final_data=pd.merge(geo_data, dfToronto, on='Neighborhood')
final_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Population_2016,Income_2016
0,M1G,Scarborough,Woburn,43.770992,-79.216917,53.485,30.878
1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,16.724,32.913
2,M2H,North York,Hillcrest Village,43.803762,-79.363452,16.934,40.442
3,M2K,North York,Bayview Village,43.786947,-79.385975,21.396,52.035
4,M4A,North York,Victoria Village,43.725882,-79.315572,17.510,35.786


In [544]:
toronto_data=geo_data[geo_data['Borough'].str.contains("")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [545]:
CLIENT_ID = 'MX1TSQG5KZAC2CC4MLN1COPPHNEMJJGVKNKVF1TEDMPW0PY5'
CLIENT_SECRET = 'RZ0Q00M1HEXMP0FPOHQ5Y24XAGFAXUCCMIOEEHXDNMEO3IMT'
VERSION = '20180604'

In [546]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=15000
    LIMIT=1000
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [547]:
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [548]:
dfToronto.head()

,Neighborhood,Population_2016,Income_2016
0,Agincourt North,29.113,30.414
1,Agincourt South-Malvern West,23.757,31.825
2,Alderwood,12.054,47.709
3,Annex,30.526,112.766
4,Banbury-Don Mills,27.695,67.757


In [549]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [550]:
df_merge = pd.merge(dfToronto, toronto_data, on='Neighborhood')

In [551]:
df_merge.head()

,Neighborhood,Population_2016,Income_2016,Postalcode,Borough,Latitude,Longitude
0,Bayview Village,21.396,52.035,M2K,North York,43.786947,-79.385975
1,Hillcrest Village,16.934,40.442,M2H,North York,43.803762,-79.363452
2,Humber Summit,12.416,30.731,M9L,North York,43.756303,-79.565963
3,Humewood-Cedarvale,14.365,65.274,M6C,York,43.693781,-79.428191
4,Scarborough Village,16.724,32.913,M1J,Scarborough,43.744734,-79.239476


In [552]:
pdf = df_merge
print ("Shape of dataset: ", pdf.shape)

pdf.head(5)

Shape of dataset:  (10, 7)


,Neighborhood,Population_2016,Income_2016,Postalcode,Borough,Latitude,Longitude
0,Bayview Village,21.396,52.035,M2K,North York,43.786947,-79.385975
1,Hillcrest Village,16.934,40.442,M2H,North York,43.803762,-79.363452
2,Humber Summit,12.416,30.731,M9L,North York,43.756303,-79.565963
3,Humewood-Cedarvale,14.365,65.274,M6C,York,43.693781,-79.428191
4,Scarborough Village,16.724,32.913,M1J,Scarborough,43.744734,-79.239476


In [553]:
import random 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs 
%matplotlib inline

In [554]:
df_new = pdf.drop(['Borough','Postalcode','Neighborhood'], axis=1)
df_new.head()

,Population_2016,Income_2016,Latitude,Longitude
0,21.396,52.035,43.786947,-79.385975
1,16.934,40.442,43.803762,-79.363452
2,12.416,30.731,43.756303,-79.565963
3,14.365,65.274,43.693781,-79.428191
4,16.724,32.913,43.744734,-79.239476


In [555]:
from sklearn.neighbors import KNeighborsClassifier

In [556]:
from sklearn.model_selection import train_test_split

In [557]:
from sklearn import preprocessing

In [558]:
X = df_new[['Latitude','Longitude','Population_2016','Income_2016']] .values 
X[0:5]

array([[43.7869473, -79.385975, '21.396', '52.035'],
       [43.8037622, -79.3634517, '16.934', '40.442'],
       [43.7563033, -79.56596329999999, '12.416', '30.731'],
       [43.6937813, -79.42819140000002, '14.365', '65.274'],
       [43.7447342, -79.23947609999999, '16.724', '32.913']], dtype=object)

In [559]:
y = df_new['Income_2016'].values
y[0:5]

array(['52.035', '40.442', '30.731', '65.274', '32.913'], dtype=object)

In [560]:
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 1.24165057, -0.17214082,  0.00417763,  0.40048429],
       [ 1.66049853,  0.03902072, -0.39842646, -0.19597948],
       [ 0.47832895, -1.85957596, -0.80608339, -0.69561372],
       [-1.07905241, -0.56793009, -0.63022607,  1.08163531],
       [ 0.19015038,  1.20132285, -0.41737466, -0.58334908]])

In [561]:
k = 2
#Train Model and Predict  
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (8, 4) (8,)
Test set: (2, 4) (2,)


In [562]:
k = 2
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [563]:
yhat = neigh.predict(X_test)
yhat[0:5]

array(['35.786', '28.875'], dtype=object)

In [564]:
from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

Train set Accuracy:  0.625
Test set Accuracy:  0.0


In [565]:
Ks = 8
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

array([0., 0., 0., 0., 0., 0., 0.])

plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()

In [566]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

The best accuracy was with 0.0 with k= 1


In [567]:
k = 1
#Train Model and Predict  
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.0, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (10, 4) (10,)
Test set: (0, 4) (0,)


In [568]:
toronto_venues = getNearbyVenues(names=toronto_data['Borough'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [570]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,900,900,900,900,900,900
Downtown Toronto,1900,1900,1900,1900,1900,1900
East Toronto,500,500,500,500,500,500
East York,500,500,500,500,500,500
Etobicoke,1200,1200,1200,1200,1200,1200
Mississauga,100,100,100,100,100,100
North York,2400,2400,2400,2400,2400,2400
Scarborough,1700,1700,1700,1700,1700,1700
West Toronto,600,600,600,600,600,600


In [572]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(10300, 168)

In [573]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Stadium,Beach,Beer Bar,Beer Store,Bookstore,Botanical Garden,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Campground,Cantonese Restaurant,Caribbean Restaurant,Castle,Chinese Restaurant,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Community Center,Concert Hall,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Liquor Store,Martial Arts School,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plaza,Pool Hall,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Road,Rock Climbing Spot,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wings Joint,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Central Toronto,0.000000,0.0000,0.012222,0.000000,0.010000,0.010000,0.015556,0.000000,0.008889,0.0,0.027778,0.020000,0.000000,0.000000,0.024444,0.000000,0.013333,0.005556,0.001111,0.018889,0.0,0.000000,0.001111,0.000000,0.000000,0.000000,0.050000,0.0,0.0,0.010000,0.010000,0.000000,0.000000,0.00,0.023333,0.020000,0.051111,0.010000,0.0,0.010000,0.008889,0.010000,0.000,0.0000,0.013333,0.008889,0.0,0.000000,0.010000,0.000000,0.000000,0.001111,0.000000,0.0,0.004444,0.0,0.007778,0.017778,0.0,0.010000,0.008889,0.0,0.0,0.000000,0.002222,0.011111,0.006667,0.000000,0.000,0.007778,0.010000,0.0,0.000000,0.000000,0.001111,0.021111,0.001111,0.000000,0.0,0.000000,0.0,0.0,0.0,0.022222,0.005556,0.0,0.0,0.000000,0.010,0.012222,0.016667,0.018889,0.010000,0.014444,0.020000,0.000000,0.0000,0.000000,0.000000,0.014444,0.000000,0.0000,0.011111,0.000000,0.010000,0.000000,0.000000,0.010000,0.010000,0.005556,0.000000,0.000000,0.0,0.010,0.007778,0.0,0.097778,0.000000,0.000000,0.0,0.011111,0.003333,0.0,0.000000,0.005556,0.010000,0.010000,0.0,0.020000,0.0,0.0,0.027778,0.000000,0.003333,0.005556,0.0,0.000000,0.0,0.000000,0.005556,0.001111,0.000000,0.000000,0.012222,0.010000,0.000000,0.010000,0.0,0.010000,0.000000,0.0000,0.010000,0.0000,0.000000,0.00,0.000000,0.000,0.002222,0.010000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,Downtown Toronto,0.000000,0.0000,0.010000,0.007368,0.010000,0.010000,0.006842,0.004211,0.000526,0.0,0.045263,0.009474,0.006316,0.001053,0.020000,0.001579,0.009474,0.000000,0.000000,0.016316,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.053684,0.0,0.0,0.008947,0.001579,0.000000,0

In [574]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [588]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(15)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,Park,Coffee Shop,Café,Bakery,Sandwich Place
1,Downtown Toronto,Park,Coffee Shop,Café,Bakery,Sandwich Place
2,East Toronto,Coffee Shop,Park,Café,Farmers Market,Brewery
3,East York,Park,Coffee Shop,Farmers Market,Café,Beach
4,Etobicoke,Park,Bakery,Café,Indian Restaurant,Restaurant
5,Mississauga,Restaurant,Bakery,Coffee Shop,Indian Restaurant,Grocery Store
6,North York,Park,Café,Coffee Shop,Grocery Store,Liquor Store
7,Scarborough,Park,Caribbean Restaurant,Coffee Shop,Beach,Grocery Store
8,West Toronto,Park,Bakery,Café,Pizza Place,Restaurant
9,York,Park,Café,Bakery,Restaurant,Pizza Place


In [576]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 3, 3, 0, 4, 1, 2, 0, 0], dtype=int32)

In [590]:
neighborhoods_venues_sorted.head(15)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1,Central Toronto,Park,Coffee Shop,Café,Bakery,Sandwich Place
1,1,Downtown Toronto,Park,Coffee Shop,Café,Bakery,Sandwich Place
2,3,East Toronto,Coffee Shop,Park,Café,Farmers Market,Brewery
3,3,East York,Park,Coffee Shop,Farmers Market,Café,Beach
4,0,Etobicoke,Park,Bakery,Café,Indian Restaurant,Restaurant
5,4,Mississauga,Restaurant,Bakery,Coffee Shop,Indian Restaurant,Grocery Store
6,1,North York,Park,Café,Coffee Shop,Grocery Store,Liquor Store
7,2,Scarborough,Park,Caribbean Restaurant,Coffee Shop,Beach,Grocery Store
8,0,West Toronto,Park,Bakery,Café,Pizza Place,Restaurant
9,0,York,Park,Café,Bakery,Restaurant,Pizza Place


In [601]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters